## How to run saved Sup3rWind models on ERA5 input data

#### The _proper_ way to run models, at least for large domains and / or time periods, is to modify the config templates in the `sup3rwind/examples/run_configs` directory and the execute the `sup3r -c config_pipeline.json pipeline` command. Here were show a simple interactive small run. The config below could be used to modify the `config_fwp_spatial.json` file. 

In [ ]:
import os
import warnings

import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from rex import init_logger

from sup3r.models import MultiStepGan as GanModel
from sup3r.pipeline.forward_pass import ForwardPass
from sup3r.pipeline.strategy import ForwardPassStrategy
from sup3r.postprocessing import CollectorNC
from sup3r.preprocessing import DataHandler
from sup3r.utilities.era_downloader import EraDownloader

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
init_logger('sup3r', log_level='DEBUG')

### 1. Load the model and check required inputs
-------------------------------------------------
Models can injest low-resolution data (`lr_features`), high-resolution "exogenous data" (`hr_exo_features`) like topography, and sparse observations (`obs_features`).

In [ ]:
model_dir = '/datasets/sup3rwind/models/sup3rwind_models_202407/'
model_dirs = [
    f'{model_dir}/sup3rwind_wind_step1_3x_1x_14f',
    f'{model_dir}/sup3rwind_wind_step2_5x_1x_14f',
]
model = GanModel.load(model_dirs)

#### Required ERA5 inputs

In [ ]:
model.lr_features

#### Required high-resolution inputs

In [ ]:
model.hr_exo_features  # this shows that topography is required at 10km and 2km resolution.

#### No observations can be ingested by this model

In [ ]:
model.obs_features

### 2. Download ERA5 input data and high-resolution exo data
------------------------------------------------------------

In [ ]:
area = [50, 22, 43, 29]  # [max_lat, min_lon, min_lat, max_lon]
pressure_levels = [800, 900, 1000]  # in hPa
monthly_fpattern = './tmp/{year}_{month}_{var}.nc'
variables = [
    'surface_pressure',
    '2m_temperature',
    'instantaneous_moisture_flux',
    'friction_velocity',
    'surface_latent_heat_flux',
    '2m_dewpoint_temperature',
    'convective_available_potential_energy',
    'k_index',
    'u',
    'v',  # this will download u / v at 10m, 100m, and the pressure levels specified above
    'zg',
    'orog',  # gepotential height at pressure levels and orography - required to derive 200m u / v
]
EraDownloader.run(
    year=2021,
    months=[1],
    area=area,
    days=[[1]],
    levels=pressure_levels,
    monthly_file_pattern=monthly_fpattern,
    variables=variables,
    max_workers=1,
    product_type='reanalysis',
)

#### The high-resolution topography can be downloaded from GTOPO30 (https://www.usgs.gov/centers/eros/science/usgs-eros-archive-digital-elevation-global-30-arc-second-elevation-gtopo30)
*Sup3r will automatically aggregate this data to the resolutions required for each model step (10km and 2km)

#### Do QA on the input data

In [ ]:
ds = DataHandler('./tmp/*.nc')
ds.qa()

### 3. Define the config for the run
-------------------------------------------
To modify the config below for a production run over a large domain and time period the `shape` and `time_slice` parameters should be changed while keeping `fwp_chunk_shape` the same or similar. `max_nodes` can also be changed to run across multiple HPC nodes.


### Optional Bias-Correction Prep
---------------------------------
Bias correction files can be created by downloading monthly average wind speeds from Vortex, through https://globalatlas.irena.org/workspace, processing these files with `sup3r.bias.bias_calc_vortex.VortexMeanPrepper`, and finally running `sup3r -c config_bias.json bias-calc` to compute the bias correction factors.

- Select the "layer" button in the IRENA workspace
- Search for "global average wind speed"
- Download all layers of the form "Global <month> average wind speed at <height> m height" with your specified lat / lon boundary
- Make sure these downloaded files have a directory structure like `./{month}/{month}_{height}m.tif`
- Run the following code to perform interpolation to the required input heights and save the output to an h5 file:

    ```
    vortex_pattern = './{month}/{month}_{height}m.tif'
    vortex_mean_file = './vortex_means.h5'

    VortexMeanPrepper.run(
        path_pattern=vortex_pattern,
        in_heights=[10, 60, 80, 100, 120, 140],
        out_heights=[10, 100, 200],
        fp_out=vortex_mean_file,
    )
    ```

- Run `sup3r -c config_bias.json bias-calc` with entries modified for your ERA5 and vortex mean file locations

In [ ]:
bc_dir = '/datasets/sup3rwind/bias_correction/ukraine'
exo_data_dir = '/datasets/sup3rwind/exogenous_data/ukraine'
era_files = './tmp/*.nc'

bc_kwargs = {}
for height in ['10m', '100m', '200m']:
    bc_kwargs[f'u_{height}'] = {
        'feature_name': f'windspeed_{height}',
        'bias_fp': f'{bc_dir}/u_{height}.h5',
        'out_range': [-100, 100],
        'temporal_avg': False,
    }
    bc_kwargs[f'v_{height}'] = {
        'feature_name': f'windspeed_{height}',
        'bias_fp': f'{bc_dir}/v_{height}.h5',
        'out_range': [-100, 100],
        'temporal_avg': False,
    }
}

config = {
    'pass_workers': 1,
    'input_handler_kwargs': {
        'time_slice': slice(
            0, 100
        ),  # time period to run (40 is a single chunk)
        'chunks': {'time': 100, 'south_north': 30, 'west_east': 30},
        'target': [
            43.0,
            22.0,
        ],  # (lat, lon) lower left coordinate of the domain to run.
        'shape': [30, 30],  # shape of domain to use (same as a single chunk)
        'cache_kwargs': {
            'cache_pattern': './cache/{feature}.nc'
        },  # where the input data will be cached
    },
    'bias_correct_method': 'monthly_local_linear_bc',
    'bias_correct_kwargs': bc_kwargs,
    'max_nodes': 1,
    'input_handler_name': 'DataHandler',
    'file_paths': era_files,
    'fwp_chunk_shape': [
        30,
        30,
        40,
    ],  # the full domain is split into chunks of this shape
    'model_class': 'MultiStepGan',
    'model_kwargs': {'model_dirs': model_dirs},
    'out_pattern': './output/ukraine_chunk_{file_id}.nc',  # where the chunks will be saved - named with time chunk index and spatial chunk index
    'spatial_pad': 5,  # spatial padding around each chunk
    'temporal_pad': 5,  # temporal padding around each chunk
    'exo_handler_kwargs': {
        'topography': {
            'chunks': {'south_north': 30, 'west_east': 30},
            'file_paths': era_files,
            'source_files': f'{exo_data_dir}/ukraine_topo.nc',
        }
    },
}

### 4. Initialize the forward pass strategy and run forward pass
-------
*This is done automatically when running `sup3r -c config_pipeline.json pipeline` according to the parameters set in `config_fwp_spatial.json`

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    strat = ForwardPassStrategy(**config)

*When the forward pass is run with `sup3r -c config_pipeline.json pipeline` something similar to the loop below will be run on N (`max_nodes`) nodes and the output will be saved to the location defined by `out_pattern`. 

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    # In this case there is only one node
    fwp = ForwardPass(strat, node_index=0)
    for idx in strat.node_chunks[0]:
        chunk = fwp.get_input_chunk(chunk_index=idx)
        failed, out = fwp.run_chunk(
            chunk=chunk,
            model_kwargs=strat.model_kwargs,
            model_class=strat.model_class,
            allowed_const=strat.allowed_const,
            output_workers=strat.output_workers,
            invert_uv=strat.invert_uv,
            nn_fill=strat.nn_fill,
            meta=fwp.meta,
        )

### 5. Collect the chunks into a single file
-----------------------
When multiple chunks are run the data collection step performs some QC and then stitches the chunks together. When running with `sup3r -c config_pipeline.json pipeline` this data collection is done automatically according to the parameters set in `config_collect_nc.json`

In [ ]:
CollectorNC.collect(
    file_paths=strat.out_files, out_file='./output/ukraine_combined.nc'
)

### 6. Visualize output 
-------------------------

#### Model outputs

In [ ]:
model.hr_out_features

In [ ]:
dh = DataHandler(
    file_paths='./output/ukraine_combined.nc', features='windspeed_100m'
)

fig, ax = plt.subplots(
    nrows=1,
    ncols=1,
    figsize=(5, 5),
    constrained_layout=True,
)

im = ax.imshow(
    dh['windspeed_100m'].isel(time=0).values,
    cmap='YlGnBu',
    origin='upper',
    clim=(0, 12),
    zorder=0,
)

fig.suptitle(
    'Wind speed over Carpathian Mountains (100m)',
    fontsize=12,
)

ax.set_xticks([])
ax.set_yticks([])
plt.colorbar(im, ax=ax, label='ws_100m', shrink=0.85)
ax.axis('off')

anim = FuncAnimation(
    fig,
    lambda frame: im.set_array(dh['windspeed_100m'].isel(time=frame).values),
    len(dh.time),
    interval=200,
    blit=False,
)

plt.close(fig)

HTML(anim.to_jshtml(fps=3, default_mode='once', embed_frames=True))